In [1]:
pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.0 MB/s eta 0:00:00


In [2]:
pip install torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.5 MB/s eta 0:00:00


In [5]:
import pandas as pd


In [6]:
pip install scikit-learn

In [7]:
df = pd.read_csv('/content/drive/MyDrive/dataset.csv', encoding='ISO-8859-1')
df.shape

half_size = len(df) // 5
trimmed_df = df.head(half_size)
trimmed_df.shape
df=trimmed_df

In [8]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((180000, 2), (20000, 2))

In [16]:
half_size = len(test_df) // 20
trimmed_df = test_df.head(half_size)
trimmed_df.shape
df=trimmed_df
df.shape

(1000, 2)

In [10]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained("grammarly/coedit-large")
model = AutoModelForSeq2SeqLM.from_pretrained("grammarly/coedit-large")

model.to('cuda')


T5ForConditionalGeneration(
  (shared): Embedding(32100, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [17]:
df.describe

<bound method NDFrame.describe of                                                     input  \
860     Amon, D. J., Zigler, A. F., Dahlgren, T. G., G...   
55284   Citibank, JP Morgan owes, Barclays Bank, Monar...   
110239          top layer here actually and extends it up   
106470  Using your current transaction Log file it and...   
63275                  At least he was held accountabel..   
...                                                   ...   
149186  The 2nd annual Oscar Peterson international ja...   
191023                       Hitoshi dont have a apetite.   
167091  In business perspective, the experiment has be...   
127096  In March 2017 same time, at, NBB filed Anti-du...   
112495  As you know DraftKings has entryinto a merger ...   

                                                   output  
860     Amon, D. J., Ziegler, A. F., Dahlgren, T. G., ...  
55284   Citibank, JP Morgan, Barclays Bank, Monarch Al...  
110239         top layer here actually and extends it

In [18]:
def correct_grammar(input_text,num_return_sequences=1):
  device = torch.device("cuda" if torch.cuda.is_available() else "cuda")
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text[0]

In [19]:
import pandas as pd

# Create a new DataFrame by applying the function to each row of the original DataFrame 'df'
new_df = df.copy()  # Copy the original DataFrame to a new one

# Define the function to correct grammar
def correct_grammar_for_row(row):
    row['output_predicted'] = correct_grammar(row['input'], num_return_sequences=1)
    return row

# Apply the function to the new DataFrame
new_df = new_df.apply(correct_grammar_for_row, axis=1)

# Print the new DataFrame
print(new_df)


                                                    input  \
860     Amon, D. J., Zigler, A. F., Dahlgren, T. G., G...   
55284   Citibank, JP Morgan owes, Barclays Bank, Monar...   
110239          top layer here actually and extends it up   
106470  Using your current transaction Log file it and...   
63275                  At least he was held accountabel..   
...                                                   ...   
149186  The 2nd annual Oscar Peterson international ja...   
191023                       Hitoshi dont have a apetite.   
167091  In business perspective, the experiment has be...   
127096  In March 2017 same time, at, NBB filed Anti-du...   
112495  As you know DraftKings has entryinto a merger ...   

                                                   output  \
860     Amon, D. J., Ziegler, A. F., Dahlgren, T. G., ...   
55284   Citibank, JP Morgan, Barclays Bank, Monarch Al...   
110239         top layer here actually and extends it up.   
106470  Using your curr

In [20]:
import pandas as pd
import Levenshtein



# Function to calculate Levenshtein distance and similarity score
def calculate_similarity(row):
    original_output = row["output"]
    predicted_output = row["output_predicted"]

    # Calculate the Levenshtein distance
    distance = Levenshtein.distance(original_output, predicted_output)

    # Calculate a similarity score (normalized Levenshtein similarity)
    max_len = max(len(original_output), len(predicted_output))
    similarity = 1 - (distance / max_len)

    return similarity

# Apply the calculate_similarity function to each row and store the result in a new column
new_df['similarity_score'] = new_df.apply(calculate_similarity, axis=1)

# Print the DataFrame with the similarity scores
print(new_df)

                                                    input  \
860     Amon, D. J., Zigler, A. F., Dahlgren, T. G., G...   
55284   Citibank, JP Morgan owes, Barclays Bank, Monar...   
110239          top layer here actually and extends it up   
106470  Using your current transaction Log file it and...   
63275                  At least he was held accountabel..   
...                                                   ...   
149186  The 2nd annual Oscar Peterson international ja...   
191023                       Hitoshi dont have a apetite.   
167091  In business perspective, the experiment has be...   
127096  In March 2017 same time, at, NBB filed Anti-du...   
112495  As you know DraftKings has entryinto a merger ...   

                                                   output  \
860     Amon, D. J., Ziegler, A. F., Dahlgren, T. G., ...   
55284   Citibank, JP Morgan, Barclays Bank, Monarch Al...   
110239         top layer here actually and extends it up.   
106470  Using your curr

In [21]:
average_similarity = new_df['similarity_score'].mean()
print(average_similarity)

0.7423457401549577
